# San Antonio 311 service calls

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## Acquire

In [17]:
df = pd.read_csv("allservicecalls.csv")

In [18]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590016 entries, 0 to 590015
Data columns (total 17 columns):
Category                590016 non-null object
CASEID                  590016 non-null int64
OPENEDDATETIME          590016 non-null object
SLA_Date                589959 non-null object
CLOSEDDATETIME          563276 non-null object
Late (Yes/No)           590016 non-null object
Dept                    589875 non-null object
REASONNAME              590016 non-null object
TYPENAME                590016 non-null object
CaseStatus              590016 non-null object
SourceID                590016 non-null object
OBJECTDESC              590016 non-null object
Council District        590016 non-null int64
XCOORD                  590016 non-null int64
YCOORD                  590016 non-null int64
Report Starting Date    590016 non-null object
Report Ending Date      590016 non-null object
dtypes: int64(4), object(13)
memory usage: 76.5+ MB


,Category,CASEID,OPENEDDATETIME,SLA_Date,CLOSEDDATETIME,Late (Yes/No),Dept,REASONNAME,TYPENAME,CaseStatus,SourceID,OBJECTDESC,Council District,XCOORD,YCOORD,Report Starting Date,Report Ending Date
0,Information,1010056445,10/18/2011,10/27/2011,11/14/2018,YES,Customer Service,311 Call Center,Customer Satisfaction,Closed,Web Portal,"19399 NW MILITARY HY, San Antonio, 78257",8,2104251,13774290,11/9/2018,11/9/2019
1,Information,1010058206,10/19/2011,10/28/2011,11/14/2018,YES,Customer Service,311 Call Center,Customer Satisfaction,Closed,Web Portal,"19399 NW MILITARY HY, San Antonio, 78257",8,2104251,13774290,11/9/2018,11/9/2019
2,Information,1010075875,10/31/2011,11/10/2011,11/14/2018,YES,Customer Service,311 Call Center,Customer Satisfaction,Closed,Web Portal,"1106 ELLISON DR S, San Antonio, 78245",4,2070537,13702644,11/9/2018,11/9/2019
3,Information,1010115259,12/1/2011,12/13/2011,11/14/2018,YES,Customer Service,311 Call Center,Customer Satisfaction,Closed,Web Portal,PLAZA DE ARMAS and COMMERCE ST W,1,2129213,13702278,11/9/2018,11/9/2019
4,Streets & Infrastructure,1010126853,12/10/2011,12/11/2011,NaN,YES,Trans & Cap Improvements,Storm Water,Tree Fallen,Open,Constituent Call,ELM CREEK and WURZBACH RD,8,2113584,13747492,11/9/2018,11/9/2019


In [14]:
data_dict = pd.read_csv("datadict.csv")
data_dict

,column,type,label,description
0,Category,text,CATEGORY,This general category was developed to place 3...
1,CASEID,text,CASEID,The unique case reference number is assigned b...
2,OPENEDDATETIME,timestamp,OPENEDDATETIME,The date and time that a case was submitted.\r\n
3,SLA_Date,timestamp,SLA_Date,Every service request type has a due date assi...
4,CLOSEDDATETIME,timestamp,CLOSEDDATETIME,The date and time that the case/request was wa...
5,Late (Yes/No),text,Late (Yes/No),This indicates whether the case has surpassed ...
6,Dept,text,Dept,The City department to whom the case is assign...
7,REASONNAME,text,REASONNAME,The department division within the City deaprt...
8,TYPENAME,text,TYPENAME,The service request type name for the issue be...
9,CaseStatus,text,CaseStatus,The status of a case which is either open or c...


In [13]:
for x  in data_dict.description:
    print(x)

This general category was developed to place 311 services in a high level category, different than their respective department.

The unique case reference number is assigned by the 311 Lagan customer relationship management system.

The date and time that a case was submitted.

Every service request type has a due date assigned to the request, based on the request type name.  The SLA Date is the due date and time for the request type based on the service level agreement (SLA).  Each service request type has a timeframe in which it is scheduled to be addressed.

The date and time that the case/request was was closed.  If blank, the request has not been closed as of the Report Ending Date.

This indicates whether the case has surpassed its Service Level Agreement due date for the specific service request.

The City department to whom the case is assigned.

The department division within the City deaprtment to whom the case is assigned.

The service request type name for the issue being r